In [ ]:
import sys
print(sys.executable)

In [ ]:
import sys
print(sys.version)

In [ ]:
!pip install fitz frontend pymupdf pypdf dotenv langchain torch langchain_community transformers datasets
!pip install "numpy<2"
!pip install --upgrade pybind11
!pip install faiss-gpu

In [1]:
import numpy as np
print(np.__version__)

1.24.4


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

# path = "C:\\KIMUJUNG\\team_project\\team_project3\\pdf_folder"
path = "pdf_folder"
file_list = os.listdir(path)
car_list = []
short_list = []
long_hurt = []
long_save = []
long_etc = []
for i in file_list:
    if i[:2] in ['개인','업무','영업','이륜','외화','농기','기타','모터','공동']:
        car_list.append(i)
    elif i[:2] == '일반':
        short_list.append(i)
    elif i[:2] == "상해":
        long_hurt.append(i)
    elif i[:2] in ['저축','연금']:
        long_save.append(i)
    elif i[:2] in ['재물','제도']:
        long_etc.append(i)
    elif i[:5] in ["운전면허교","운전자_l"]:
        car_list.append(i)
    elif i[:5] == "운전자_(":
        long_etc.append(i)

len(file_list), len(car_list)+len(short_list)+len(long_hurt)+len(long_save)+len(long_etc)

(141, 141)

In [4]:
len(car_list),len(short_list),len(long_hurt),len(long_save),len(long_etc)

(23, 55, 41, 3, 19)

In [5]:
import faiss
import pickle
import numpy as np
import torch
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import fitz  # PyMuPDF
import re

# 1️⃣ GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/kwj/team_proj/faiss_2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


자동차 보험

In [6]:
# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)  # 페이지 번호 제거
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)  # 목차 제거
    text = re.sub(r"금소법|법령", "", text)  # 법적 공지사항 제거
    text = re.sub(r"\s{2,}", " ", text)  # 공백 정리
    return text.strip()

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index_lotte_car.bin"
metadata_path = "./documents_lotte_car.pkl"

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

all_documents = []
for file in car_list:
    all_texts = []
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata  # PDF의 메타데이터 추출

            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)  # ✅ 전체 텍스트 리스트에 저장

    # 7️⃣ 텍스트 병합 후 중복 제거
    total_text = "\n".join(all_texts)  # ✅ 하나의 문자열로 병합
    unique_texts = list(dict.fromkeys(total_text.split("\n")))  # ✅ 중복 제거 후 리스트 변환

    # 8️⃣ 새로운 페이지 번호 추가하며 Document 형식 변환
    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)  # ✅ 중복 제거된 문서 사용

    # 🔟 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]

    embeddings = []  # ✅ 리스트 초기화
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)  # ✅ 리스트에 추가

    embeddings = np.vstack(embeddings).astype(np.float32)  # ✅ 배열 변환

    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환

    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)
    
    print(f"✅ {file} 처리 완료")

faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

print("🎉 모든 문서 처리 및 저장 완료!")

✅ 공동인수_영업용 자동차보험(공동).pdf 처리 완료
✅ 이륜차_let way 이륜자동차보험.pdf 처리 완료
✅ 개인용_let click 개인용자동차보험(인터넷).pdf 처리 완료
✅ 업무용_let way 업무용법인소유자동차보험.pdf 처리 완료
✅ 기타_let way 자동차취급업자종합보험.pdf 처리 완료
✅ 운전면허교습생_let way 운전면허교습생자동차보험.pdf 처리 완료
✅ 공동인수_업무용 자동차보험(공동).pdf 처리 완료
✅ 개인용_let way 개인용자동차보험.pdf 처리 완료
✅ 업무용_let way 업무용개인소유자동차보험.pdf 처리 완료
✅ 업무용_let click 업무용법인소유자동차보험(인터넷).pdf 처리 완료
✅ 농기계_let way 농기계보험.pdf 처리 완료
✅ 운전자_let way 운전자보험.pdf 처리 완료
✅ 공동인수_개인용 자동차보험(공동).pdf 처리 완료
✅ 기타_let way 배달플랫폼자동차보험.pdf 처리 완료
✅ 업무용_let click 업무용개인소유자동차보험(TM).pdf 처리 완료
✅ 영업용_let click 영업용자동차보험(TM).pdf 처리 완료
✅ 업무용_let click 업무용개인소유자동차보험(인터넷).pdf 처리 완료
✅ 기타_CREW 원데이자동차보험.pdf 처리 완료
✅ 개인용_let click 개인용자동차보험(TM).pdf 처리 완료
✅ 업무용_let click 업무용법인소유자동차보험(TM).pdf 처리 완료
✅ 공동인수_이륜자동차보험(공동).pdf 처리 완료
✅ 영업용_let way 영업용자동차보험.pdf 처리 완료
✅ 기타_let click 배달플랫폼자동차보험(인터넷).pdf 처리 완료
🎉 모든 문서 처리 및 저장 완료!


일반 보험

In [ ]:
# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)  # 페이지 번호 제거
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)  # 목차 제거
    text = re.sub(r"금소법|법령", "", text)  # 법적 공지사항 제거
    text = re.sub(r"\s{2,}", " ", text)  # 공백 정리
    return text.strip()

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index_lotte_short.bin"
metadata_path = "./documents_lotte_short.pkl"

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

all_documents = []
for file in short_list:
    all_texts = []
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata  # PDF의 메타데이터 추출

            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)  # ✅ 전체 텍스트 리스트에 저장

    # 7️⃣ 텍스트 병합 후 중복 제거
    total_text = "\n".join(all_texts)  # ✅ 하나의 문자열로 병합
    unique_texts = list(dict.fromkeys(total_text.split("\n")))  # ✅ 중복 제거 후 리스트 변환

    # 8️⃣ 새로운 페이지 번호 추가하며 Document 형식 변환
    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)  # ✅ 중복 제거된 문서 사용

    # 🔟 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]

    embeddings = []  # ✅ 리스트 초기화
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)  # ✅ 리스트에 추가

    embeddings = np.vstack(embeddings).astype(np.float32)  # ✅ 배열 변환

    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환

    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)
    print(f"✅ {file} 처리 완료")

# 1️⃣3️⃣ FAISS 인덱스 저장
faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

장기 - 상해, 질병

In [ ]:
# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)  # 페이지 번호 제거
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)  # 목차 제거
    text = re.sub(r"금소법|법령", "", text)  # 법적 공지사항 제거
    text = re.sub(r"\s{2,}", " ", text)  # 공백 정리
    return text.strip()

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index_lotte_long_hurt.bin"
metadata_path = "./documents_lotte_long_hurt.pkl"

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

all_documents = []
for file in long_hurt:
    all_texts = []
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata  # PDF의 메타데이터 추출

            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)  # ✅ 전체 텍스트 리스트에 저장

    # 7️⃣ 텍스트 병합 후 중복 제거
    total_text = "\n".join(all_texts)  # ✅ 하나의 문자열로 병합
    unique_texts = list(dict.fromkeys(total_text.split("\n")))  # ✅ 중복 제거 후 리스트 변환

    # 8️⃣ 새로운 페이지 번호 추가하며 Document 형식 변환
    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)  # ✅ 중복 제거된 문서 사용

    # 🔟 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]

    embeddings = []  # ✅ 리스트 초기화
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)  # ✅ 리스트에 추가

    embeddings = np.vstack(embeddings).astype(np.float32)  # ✅ 배열 변환

    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환

    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)
        
    print(f"✅ {file} 처리 완료")

faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

print("🎉 모든 문서 처리 및 저장 완료!")

장기 - 연금, 저축

In [6]:
# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)  # 페이지 번호 제거
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)  # 목차 제거
    text = re.sub(r"금소법|법령", "", text)  # 법적 공지사항 제거
    text = re.sub(r"\s{2,}", " ", text)  # 공백 정리
    return text.strip()

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index_lotte_long_save.bin"
metadata_path = "./documents_lotte_long_save.pkl"

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

all_documents = []
for file in long_save:
    all_texts = []
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata  # PDF의 메타데이터 추출

            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)  # ✅ 전체 텍스트 리스트에 저장

    # 7️⃣ 텍스트 병합 후 중복 제거
    total_text = "\n".join(all_texts)  # ✅ 하나의 문자열로 병합
    unique_texts = list(dict.fromkeys(total_text.split("\n")))  # ✅ 중복 제거 후 리스트 변환

    # 8️⃣ 새로운 페이지 번호 추가하며 Document 형식 변환
    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)  # ✅ 중복 제거된 문서 사용

    # 🔟 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]

    embeddings = []  # ✅ 리스트 초기화
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)  # ✅ 리스트에 추가

    embeddings = np.vstack(embeddings).astype(np.float32)  # ✅ 배열 변환

    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환

    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)    
    print(f"✅ {file} 처리 완료")

faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

print("🎉 모든 문서 처리 및 저장 완료!")

✅ 연금보험_let care 연금저축손해보험Ⅰ(2401).pdf 처리 완료
✅ 연금보험_let care 연금저축손해보험Ⅱ(2401).pdf 처리 완료
✅ 연금보험_let care 연금저축손해보험(계약이전)(2401).pdf 처리 완료
🎉 모든 문서 처리 및 저장 완료!


장기 - 기타

In [ ]:
# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"p\.\d+", "", text)  # 페이지 번호 제거
    text = re.sub(r"(제작일|주소|QR코드|MEMO).*?(\n|$)", "", text, flags=re.DOTALL)  # 목차 제거
    text = re.sub(r"금소법|법령", "", text)  # 법적 공지사항 제거
    text = re.sub(r"\s{2,}", " ", text)  # 공백 정리
    return text.strip()

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index_lotte_long_etc.bin"
metadata_path = "./documents_lotte_long_etc.pkl"

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

all_documents = []
for file in long_etc:
    all_texts = []
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata  # PDF의 메타데이터 추출

            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
        doc.metadata["source"] = file
        all_texts.append(doc.page_content)  # ✅ 전체 텍스트 리스트에 저장

    # 7️⃣ 텍스트 병합 후 중복 제거
    total_text = "\n".join(all_texts)  # ✅ 하나의 문자열로 병합
    unique_texts = list(dict.fromkeys(total_text.split("\n")))  # ✅ 중복 제거 후 리스트 변환

    # 8️⃣ 새로운 페이지 번호 추가하며 Document 형식 변환
    new_documents = []
    for i, text in enumerate(unique_texts):
        new_doc = Document(page_content=text, metadata={"page": i + 1})
        new_documents.append(new_doc)

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(new_documents)  # ✅ 중복 제거된 문서 사용

    # 🔟 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]

    embeddings = []  # ✅ 리스트 초기화
    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)  # ✅ 리스트에 추가

    embeddings = np.vstack(embeddings).astype(np.float32)  # ✅ 배열 변환

    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환

    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)    
    print(f"✅ {file} 처리 완료")

faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

print("🎉 모든 문서 처리 및 저장 완료!")

In [ ]:
# 1️⃣ 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index.bin")

# 2️⃣ 문서 정보 로드
with open("./documents.pkl", "rb") as f:
    documents = pickle.load(f)

# 3️⃣ 검색 수행
# query = " 보험금의 종류 및 한도에 대해 설명해줘?"
query = "자동차 의무보험 미가입에 따른 불이익을 알려줘"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=10)  # 가장 유사한 5개 검색
context = []

# 4️⃣ 검색 결과 출력
for idx in I[0]:
    context.append(documents[idx].page_content)
    print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")


[]